<a href="https://colab.research.google.com/github/SeongUgKim/gender_bias_in_nlp/blob/main/MBE_Calcuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import torch
import difflib
import nltk
import regex as re
import numpy as np
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [ ]:
def load_tokenizer_and_model(lang):
    if lang == 'de':
        model_name = 'deepset/gbert-base'
    elif lang == 'es':
        model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
    elif lang == 'pt':
        model_name = 'neuralmind/bert-base-portuguese-cased'
    elif lang == 'en':
        model_name = 'bert-base-cased'
    elif lang == 'zh':
        model_name = 'hfl/chinese-bert-wwm-ext'

    model = AutoModelForMaskedLM.from_pretrained(model_name, output_hidden_states=True, output_attentions=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = model.eval()
    if torch.cuda.is_available():
        model.to('cuda')
    return tokenizer, model

In [ ]:
def read_list(filename):
    with open(filename, 'rb') as f:
        n_list = pickle.load(f, encoding='utf8')
        return n_list

In [ ]:
def calculate_aul(model, token_ids, log_softmax, attention):
    output = model(token_ids)
    logits = output.logits.squeeze(0)
    log_probs = log_softmax(logits)
    token_ids = token_ids.view(-1, 1).detach()
    token_log_probs = log_probs.gather(1, token_ids)[1:-1]
    if attention:
        attentions = torch.mean(torch.cat(output.attentions, 0), 0)
        averaged_attentions = torch.mean(attentions, 0)
        averaged_token_attentions = torch.mean(averaged_attentions, 0)
        token_log_probs = token_log_probs.squeeze(1) * averaged_token_attentions[1:-1]
    sentence_log_prob = torch.mean(token_log_probs)
    score = sentence_log_prob.item()
    return score

In [ ]:
def calculate_mbe(lang, male_filepath, female_filepath, male_list, female_list):
    tokenizer, model = load_tokenizer_and_model(lang)
    total_score = 0
    stereo_score = 0
    if torch.cuda.is_available():
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    masked_id = tokenizer.mask_token_id
    log_softmax = torch.nn.LogSoftmax(dim=1)
    male = read_list(male_filepath) if male_filepath is not None else male_list
    female = read_list(female_filepath) if female_filepath is not None else female_list
    male_inputs = [tokenizer.encode(sentence, return_tensors='pt') for sentence in male]
    female_inputs = [tokenizer.encode(sentence, return_tensors='pt') for sentence in female]
    attention = True
    female_scores = []
    male_scores = []

    for female_tokens in female_inputs:
        with torch.no_grad():
            female_score = calculate_aul(model, female_tokens, log_softmax, attention)
            female_scores.append(female_score)
    for male_tokens in male_inputs:
        with torch.no_grad():
            male_score = calculate_aul(model, male_tokens, log_softmax, attention)
            male_scores.append(male_score)

    female_scores = np.array(female_scores)
    male_scores = np.array(male_scores)
    bias_scores = male_scores > female_scores
    biasRating = np.sum(bias_scores).item()
    total_sentences = (bias_scores.shape[0])
    MBE = biasRating / total_sentences
    return round(MBE * 100, 2)

In [ ]:
def cos_sim(v1, v2):
  return np.dot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
def calculate_aul_kaneko(model, token_ids, log_softmax, attention):
  output = model(token_ids)
  logits = output.logits.squeeze(0)
  log_probs = log_softmax(logits)
  token_ids = token_ids.view(-1,1).detach()
  token_log_probs = log_probs.gather(1, token_ids)[1:-1]
  if attention:
      attentions = torch.mean(torch.cat(output.attentions, 0), 0)
      averaged_attentions = torch.mean(attentions, 0)
      averaged_token_attentions = torch.mean(averaged_attentions, 0)
      token_log_probs = token_log_probs.squeeze(1) * averaged_token_attentions[1:-1]
  sentence_log_prob = torch.mean(token_log_probs)
  score = sentence_log_prob.item()
  hidden_states = output.hidden_states[-1][:,1:-1]
  hidden_state = torch.mean(hidden_states, 1).detach().cpu().numpy()
  return score, hidden_state


In [ ]:
def calculate_mbe_kaneko(lang, male_filepath, female_filepath, male_list, female_list):
    if lang == 'de':
        model_name = 'deepset/gbert-base'
    elif lang == 'es':
        model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
    elif lang == 'pt':
        model_name = 'neuralmind/bert-base-portuguese-cased'
    elif lang == 'en':
        model_name = 'bert-base-cased'
    elif lang == 'zh':
        model_name = 'hfl/chinese-bert-wwm-ext'
    tokenizer, model = load_tokenizer_and_model(lang)
    total_score = 0
    stereo_score = 0
    tokenizer2 = BertTokenizer.from_pretrained(model_name)
    if torch.cuda.is_available():
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    masked_id = tokenizer.mask_token_id
    log_softmax = torch.nn.LogSoftmax(dim=1)
    male = read_list(male_filepath) if male_filepath is not None else male_list
    female = read_list(female_filepath) if female_filepath is not None else female_list
    male_inputs = [tokenizer.encode(sentence, return_tensors='pt') for sentence in male if len(tokenizer2.tokenize(sentence)) < 512]
    female_inputs = [tokenizer.encode(sentence, return_tensors='pt') for sentence in female if len(tokenizer2.tokenize(sentence)) < 512]
    attention = True
    female_scores = []
    male_scores = []
    female_embes = []
    male_embes = []

    for female_tokens in female_inputs:
        with torch.no_grad():
            female_score, female_hidden_state = calculate_aul_kaneko(model, female_tokens, log_softmax, attention)
            female_scores.append(female_score)
            female_embes.append(female_hidden_state)
    for male_tokens in male_inputs:
        with torch.no_grad():
            male_score, male_hidden_state = calculate_aul_kaneko(model, male_tokens, log_softmax, attention)
            male_scores.append(male_score)
            male_embes.append(male_hidden_state)

    female_scores = np.array(female_scores)
    female_scores = female_scores.reshape([-1,1])
    male_scores = np.array(male_scores)
    male_scores = male_scores.reshape([-1,1])
    if len(male_scores) > len(female_scores):
      male_scores = male_scores[0:len(female_scores)]
    else:
      female_scores = female_scores[0:len(male_scores)]
    bias_scores = male_scores > female_scores
    female_embes = np.concatenate(female_embes)
    male_embes = np.concatenate(male_embes)
    weights = cos_sim(female_embes, male_embes.T)
    weighted_bias_scores = bias_scores * weights
    MBE = np.sum(weighted_bias_scores) / np.sum(weights)
    return round(MBE * 100, 2)

In [ ]:
from transformers import BertTokenizer

In [ ]:
def preprocess(lang, filepath):
    if lang == 'de':
        model = 'deepset/gbert-base'
    elif lang == 'es':
        model = 'dccuchile/bert-base-spanish-wwm-uncased'
    elif lang == 'pt':
        model = 'neuralmind/bert-base-portuguese-cased'
    elif lang == 'en':
        model = 'bert-base-cased'
    elif lang == 'zh':
        model = 'hfl/chinese-bert-wwm-ext'

    orignial_list = read_list(filepath)
    tokenizer = BertTokenizer.from_pretrained(model)
    result = [sentence for sentence in orignial_list if len(tokenizer.tokenize(sentence)) < 512]
    return result

In [ ]:
calculate_mbe('de', None, None, rule_male_de, rule_female_de)

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


48.64